In [ ]:
!pip install fastapi uvicorn scikit-learn matplotlib seaborn pyngrok streamlit nest-asyncio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from fastapi import FastAPI, HTTPException, Query
from typing import Optional
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import uvicorn
import threading

In [ ]:
from pathlib import Path

file_path = Path("/content/dataset.csv")

try:
    data = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found.")
    exit()

In [ ]:
element_mapping = {
    "Emissions Share (CH4)": "Emissions_Share_CH4",
    "Emissions Share (CO2)": "Emissions_Share_CO2",
    "Emissions Share (CO2eq) (AR5)": "Emissions_Share_CO2eq_AR5",
    "Emissions Share (CO2eq) (AR5) (F-gases)": "Emissions_Share_CO2eq_AR5_F_gases",
    "Emissions Share (N2O)": "Emissions_Share_N2O",
    "Emissions per area of agricultural land": "Emissions_per_area",
    "Emissions per capita": "Emissions_per_capita",
    "Emissions per value of agricultural production": "Emissions_per_value",
}

In [ ]:
print(data.columns)

Index(['Domain Code', 'Domain', 'Area Code (M49)', 'Area', 'Element Code',
       'Element', 'Item Code', 'Item', 'Year Code', 'Year', 'Unit', 'Value',
       'Flag', 'Flag Description'],
      dtype='object')


In [ ]:
print(data.head())

  Domain Code                Domain  Area Code (M49)   Area  Element Code  \
0          EM  Emissions indicators              356  India        726313   
1          EM  Emissions indicators              356  India        726313   
2          EM  Emissions indicators              356  India        726313   
3          EM  Emissions indicators              356  India        726313   
4          EM  Emissions indicators              356  India        726313   

                         Element  Item Code       Item  Year Code  Year Unit  \
0  Emissions Share (CO2eq) (AR5)       6996  Farm gate       1990  1990    %   
1  Emissions Share (CO2eq) (AR5)       6996  Farm gate       1991  1991    %   
2  Emissions Share (CO2eq) (AR5)       6996  Farm gate       1992  1992    %   
3  Emissions Share (CO2eq) (AR5)       6996  Farm gate       1993  1993    %   
4  Emissions Share (CO2eq) (AR5)       6996  Farm gate       1994  1994    %   

   Value Flag Flag Description  
0  51.99    E  Estimate

In [ ]:
data = data.groupby(['Area', 'Year', 'Element'])['Value'].mean().unstack().reset_index()
data.rename(columns=element_mapping, inplace=True)

In [ ]:
data.dropna(inplace=True)

In [ ]:
print(data)

Element   Area  Year  Emissions_Share_CH4  Emissions_Share_CO2  \
0        India  1990            41.741250            29.370833   
1        India  1991            41.655625            29.525833   
2        India  1992            41.632500            29.485833   
3        India  1993            41.785000            29.737500   
4        India  1994            41.705625            29.705833   
5        India  1995            41.503750            29.770000   
6        India  1996            41.460000            29.563333   
7        India  1997            41.231250            29.645833   
8        India  1998            41.246875            29.655833   
9        India  1999            41.107500            29.282500   
10       India  2000            41.029375            29.199167   
11       India  2001            40.987500            28.823333   
12       India  2002            40.828125            28.975833   
13       India  2003            40.655000            29.064167   
14       I

In [ ]:
numeric_cols = list(element_mapping.values())

for col in numeric_cols:
    if data[col].dtype != 'number':
        try:
            data[col] = pd.to_numeric(data[col], errors='coerce')

        except Exception as e:
            print(f"Error converting column '{col}' to numeric: {e}")

for col in numeric_cols:
    if data[col].isnull().any():
        mean_value = data[col].mean()
        data[col].fillna(mean_value, inplace=True)

In [ ]:
data['Total_Emissions_Share'] = data[[col for col in data.columns if col.startswith('Emission_Share_')]].sum(axis=1)

In [ ]:
X = data.drop(["Emissions_per_area", "Area", "Year"], axis=1)
y = 1 / (data["Emissions_per_area"] + 1e-9)

In [ ]:
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = [col for col in X.columns if col not in numerical_features]

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough' #Handles any unexpected column more robustly
)

In [ ]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', RandomForestRegressor(random_state=42))])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE : {mse} ")
print(f"r2 score : {r2}")

MSE : 1.0402906987608268e-05 
r2 score : 0.9606877851486674


In [ ]:
try:
    feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()
except AttributeError:
    try:
        feature_names = pipeline.named_steps['preprocessor'].get_feature_names()
    except AttributeError:
        print("Warning: Could not retrieve feature names.  Check your scikit-learn version and preprocessor type.")
        feature_names = None

if feature_names is not None:
    importances = pipeline.named_steps['regressor'].feature_importances_
    feature_importances_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)

In [ ]:
print("\n Feature Importances:")
print(feature_importances_df)


 Feature Importances:
                                  Feature  Importance
5               num__Emissions_per_capita    0.260900
2          num__Emissions_Share_CO2eq_AR5    0.253911
6                num__Emissions_per_value    0.171642
0                num__Emissions_Share_CH4    0.137198
1                num__Emissions_Share_CO2    0.103669
4                num__Emissions_Share_N2O    0.047022
3  num__Emissions_Share_CO2eq_AR5_F_gases    0.025658
7              num__Total_Emissions_Share    0.000000


In [ ]:
app = FastAPI(title="Carbon Credits and Emissions Prediction API")

In [ ]:
@app.get("/predict_emissions")
async def predict_emissions(
    area: str = Query(..., description="Area (e.g., country)"),
    year: int = Query(..., description="Year"),
    ch4_emissions_share: float = Query(..., description="Emissions Share (CH4)"),
    co2_emissions_share: float = Query(..., description="Emissions Share (CO2)"),
    co2eq_ar5_emissions_share: float = Query(..., description="Emissions Share (CO2eq) (AR5)"),
    co2eq_ar5_fgases_emissions_share: float = Query(..., description="Emissions Share (CO2eq) (AR5) (F-gases)"),
    n2o_emissions_share: float = Query(..., description="Emissions Share (N2O)"),
    emissions_per_capita: float = Query(..., description="Emissions per capita"),
    emissions_per_value: float = Query(..., description="Emissions per value of agricultural production"),
):
    """Predicts emissions per area using the trained model."""

    try:
        future_data = pd.DataFrame({
            'Area': [area],
            'Year': [year],
            'Emissions_Share_CH4': [ch4_emissions_share],
            'Emissions_Share_CO2': [co2_emissions_share],
            'Emissions_Share_CO2eq_AR5': [co2eq_ar5_emissions_share],
            'Emissions_Share_CO2eq_AR5_F_gases': [co2eq_ar5_fgases_emissions_share],
            'Emissions_Share_N2O': [n2o_emissions_share],
            'Emissions_per_capita': [emissions_per_capita],
            'Emissions_per_value': [emissions_per_value],
        })

        future_data['Total_Emissions_Share'] = future_data[[col for col in future_data.columns if col.startswith('Emissions_Share_')]].sum(axis=1)

        X_future = future_data.drop(['Area', 'Year'], axis=1)  # Drop target

        X_future_transformed = pipeline.named_steps['preprocessor'].transform(X_future)

        feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()
        X_future_transformed = pd.DataFrame(X_future_transformed, columns=feature_names)

        future_predictions = pipeline.predict(X_future)
        predicted_emissions_per_area = 1 / (future_predictions + 1e-9)

        return {
            "area": area,
            "year": year,
            "predicted_emissions_per_area": predicted_emissions_per_area[0],
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"An error occurred: {e}")



@app.get("/calculate_carbon_credits")
async def calculate_carbon_credits(
    area: str = Query(..., description="Area"),
    year: int = Query(..., description="Year"),
    ch4_emissions_share: float = Query(..., description="Emissions Share (CH4)"),
    co2_emissions_share: float = Query(..., description="Emissions Share (CO2)"),
    co2eq_ar5_emissions_share: float = Query(..., description="Emissions Share (CO2eq) (AR5)"),
    co2eq_ar5_fgases_emissions_share: float = Query(..., description="Emissions Share (CO2eq) (AR5) (F-gases)"),
    n2o_emissions_share: float = Query(..., description="Emissions Share (N2O)"),
    emissions_per_capita: float = Query(..., description="Emissions per capita"),
    emissions_per_value: float = Query(..., description="Emissions per value of agricultural production"),
    baseline_emissions_per_area: Optional[float] = Query(None, description="Baseline emissions per area (optional)"),
    reduction_achieved: Optional[float] = Query(None, description="Emission reduction achieved (optional)")

):

    try:
        # 1. Get predicted emissions per area (same logic as in /predict_emissions)
        future_data = pd.DataFrame({  # Create the input DataFrame
            'Area': [area],
            'Year': [year],
            'Emissions_Share_CH4': [ch4_emissions_share],
            'Emissions_Share_CO2': [co2_emissions_share],
            'Emissions_Share_CO2eq_AR5': [co2eq_ar5_emissions_share],
            'Emissions_Share_CO2eq_AR5_F_gases': [co2eq_ar5_fgases_emissions_share],
            'Emissions_Share_N2O': [n2o_emissions_share],
            'Emissions_per_capita': [emissions_per_capita],
            'Emissions_per_value': [emissions_per_value],
        })
        future_data['Total_Emissions_Share'] = future_data[[col for col in future_data.columns if col.startswith('Emissions_Share_')]].sum(axis=1)

        X_future = future_data.drop(['Area', 'Year'], axis=1)
        X_future_transformed = pipeline.named_steps['preprocessor'].transform(X_future)
        feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()
        X_future_transformed = pd.DataFrame(X_future_transformed, columns=feature_names)
        future_predictions = pipeline.predict(X_future)
        predicted_emissions_per_area = 1 / (future_predictions + 1e-9)
        predicted_emissions_per_area = predicted_emissions_per_area[0]


        # 2. Calculate carbon credits
        if baseline_emissions_per_area is None:
            baseline_emissions_per_area = predicted_emissions_per_area

        if reduction_achieved is not None:
            carbon_credits = reduction_achieved
        else:
            carbon_credits = baseline_emissions_per_area - predicted_emissions_per_area

        if carbon_credits < 0:
            carbon_credits = 0

        return {
            "area": area,
            "year": year,
            "predicted_emissions_per_area": predicted_emissions_per_area,
            "baseline_emissions_per_area": baseline_emissions_per_area,
            "potential_carbon_credits": carbon_credits,
            "message": "Carbon credit calculation (simplified example). See documentation for important considerations.",
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"An error occurred: {e}")

In [ ]:
st.title ("Carbon Credit and Emissions Prediction App")

st.sidebar.header("Input Parameters")
area = st.sidebar.text_input("Area (eg: Your Country)")
year = st.sidebar.number_input("Year", min_value=1900, max_value=2100, value=2025)
ch4_emissions_share = st.sidebar.number_input("Emissions Share (CH4)", min_value=0.0, value=0.0)
co2_emissions_share = st.sidebar.number_input("Emissions Share (CO2)", min_value=0.0, value=0.0)
co2eq_ar5_emissions_share = st.sidebar.number_input("Emissions Share (CO2eq) (AR5)", min_value=0.0, value=0.0)
co2eq_ar5_fgases_emissions_share = st.sidebar.number_input("Emissions Share (CO2eq) (AR5) (F-gases)", min_value=0.0, value=0.0)
n2o_emissions_share = st.sidebar.number_input("Emissions Share (N2O)", min_value=0.0, value=0.0)
emissions_per_capita = st.sidebar.number_input("Emissions per capita", min_value=0.0, value=0.0)
emissions_per_value = st.sidebar.number_input("Emissions per value of agricultural production", min_value=0.0, value=0.0)
baseline_emissions_per_area = st.sidebar.number_input("Baseline Emissions per Area (optional)", min_value=0.0, value=0.0)
reduction_achieved = st.sidebar.number_input("Reduction Achieved (optional)", min_value=0.0, value=0.0)

if st.sidebar.button("Predict and Calculate"):
    if not area:
        st.error("Please Provide Area.")
    else:
        try:
            import requests
            prediction_url = f"{ngrok_url}/prediction_emissions?area={area}&year={year}&ch4_emissions_share={ch4_emissions_share}&co2_emissions_share={co2_emissions_share}&co2eq_ar5_emissions_share={co2eq_ar5_emissions_share}&co2eq_ar5_fgases_emissions_share={co2eq_ar5_fgases_emissions_share}&n2o_emissions_share={n2o_emissions_share}&emissions_per_capita={emissions_per_capita}&emissions_per_value={emissions_per_value}"
            predict_response = requests.get(predict_url).json()

            calculate_url = f"{ngrok_url}/calculate_carbon_credits?area={area}&year={year}&ch4_emissions_share={ch4_emissions_share}&co2_emissions_share={co2_emissions_share}&co2eq_ar5_emissions_share={co2eq_ar5_emissions_share}&co2eq_ar5_fgases_emissions_share={co2eq_ar5_fgases_emissions_share}&n2o_emissions_share={n2o_emissions_share}&emissions_per_capita={emissions_per_capita}&emissions_per_value={emissions_per_value}&baseline_emissions_per_area={baseline_emissions_per_area}&reduction_achieved={reduction_achieved}"
            calculate_response = requests.get(calculate_url).json()

            st.subheader("Results")
            st.write(f"**Area:** {calculate_response['area']}")
            st.write(f"**Year:** {calculate_response['year']}")
            st.write(f"**Predicted Emissions per Area:** {calculate_response['predicted_emissions_per_area']:.2f}")
            st.write(f"**Baseline Emissions per Area:** {calculate_response['baseline_emissions_per_area']:.2f}")
            st.write(f"**Potential Carbon Credits:** {calculate_response['potential_carbon_credits']:.2f}")
            st.write(calculate_response['message'])

        except requests.exceptions.RequestException as e:
            st.error(f"Error communicating with the API: {e}")
        except Exception as e:
            st.error(f"An error occurred: {e}")

2025-02-19 18:07:54.771 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 18:07:55.015 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-02-19 18:07:55.016 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 18:07:55.023 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 18:07:55.024 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 18:07:55.025 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 18:07:55.027 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 18:07:55.028 Thread 'MainThread': mi

In [ ]:
# Update the ngrok and FastAPI setup section
from pyngrok import ngrok, conf
import nest_asyncio
import streamlit as st
import time

# Apply nest_asyncio
nest_asyncio.apply()

# Initialize FastAPI app (keep your existing FastAPI code)
app = FastAPI(title="Carbon Credits and Emissions Prediction API")

def setup_ngrok():
    try:
        # Kill any existing ngrok processes
        ngrok.kill()

        # Configure ngrok
        NGROK_AUTH_TOKEN = "2tFWCFyzMF9XGhYpgX1EWi9ntkg_65VBoAWGSCMToQLLBn6je"
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)

        # Set up a specific configuration for the tunnel
        conf.get_default().config_path = None

        # Start ngrok with explicit configuration
        tunnel = ngrok.connect(addr="8000", proto="http", bind_tls=True)
        time.sleep(3)  # Give ngrok a moment to establish the tunnel

        return tunnel.public_url
    except Exception as e:
        print(f"Error setting up ngrok: {e}")
        return None

def run_fastapi():
    config = uvicorn.Config(
        app=app,
        host="127.0.0.1",  # Local host
        port=8000,
        log_level="info",
        loop="asyncio"
    )
    server = uvicorn.Server(config)
    try:
        server.run()
    except Exception as e:
        print(f"Error running FastAPI server: {e}")

if __name__ == "__main__":
    # Start FastAPI in a separate thread
    fastapi_thread = threading.Thread(target=run_fastapi, daemon=True)
    fastapi_thread.start()
    time.sleep(3)  # Give FastAPI server time to start

    # Setup ngrok
    ngrok_url = setup_ngrok()
    if not ngrok_url:
        print("Failed to establish ngrok connection")
        exit()

    print(f"Public URL: {ngrok_url}")

    # Your existing Streamlit code with updated API calls
    st.title("Carbon Credit and Emissions Prediction App")

    if st.sidebar.button("Predict and Calculate"):
        if not area:
            st.error("Please Provide Area.")
        else:
            try:
                # Create session for requests
                session = requests.Session()

                # Prepare parameters
                params = {
                    "area": area,
                    "year": year,
                    "ch4_emissions_share": ch4_emissions_share,
                    "co2_emissions_share": co2_emissions_share,
                    "co2eq_ar5_emissions_share": co2eq_ar5_emissions_share,
                    "co2eq_ar5_fgases_emissions_share": co2eq_ar5_fgases_emissions_share,
                    "n2o_emissions_share": n2o_emissions_share,
                    "emissions_per_capita": emissions_per_capita,
                    "emissions_per_value": emissions_per_value
                }

                # Make API calls with proper error handling
                try:
                    prediction_response = session.get(f"{ngrok_url}/predict_emissions", params=params, timeout=10)
                    prediction_response.raise_for_status()
                    predict_data = prediction_response.json()

                    # Add additional parameters for carbon credits
                    params.update({
                        "baseline_emissions_per_area": baseline_emissions_per_area,
                        "reduction_achieved": reduction_achieved
                    })

                    credits_response = session.get(f"{ngrok_url}/calculate_carbon_credits", params=params, timeout=10)
                    credits_response.raise_for_status()
                    credits_data = credits_response.json()

                    # Display results
                    st.subheader("Results")
                    st.write(f"**Area:** {credits_data['area']}")
                    st.write(f"**Year:** {credits_data['year']}")
                    st.write(f"**Predicted Emissions per Area:** {credits_data['predicted_emissions_per_area']:.2f}")
                    st.write(f"**Baseline Emissions per Area:** {credits_data['baseline_emissions_per_area']:.2f}")
                    st.write(f"**Potential Carbon Credits:** {credits_data['potential_carbon_credits']:.2f}")
                    st.write(credits_data['message'])

                except requests.exceptions.RequestException as e:
                    st.error(f"API Request Error: {str(e)}")

            except Exception as e:
                st.error(f"An unexpected error occurred: {str(e)}")
                print(f"Error details: {e}")

# Clean up ngrok on script exit
def cleanup():
    try:
        ngrok.kill()
    except:
        pass

import atexit
atexit.register(cleanup)


INFO:     Started server process [736]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


2025-02-19 18:08:03.976 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 18:08:03.978 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 18:08:03.982 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 18:08:03.983 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 18:08:03.984 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 18:08:03.986 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-19 18:08:03.987 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Public URL: https://df8c-35-197-21-93.ngrok-free.app


<function __main__.cleanup()>